## Imports

In [ ]:
import datetime
import builtins
from functools import reduce
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *

## Query A

In [ ]:
spark = SparkSession.builder.master('local[*]').appName('routes').getOrCreate()
sc = spark.sparkContext

In [ ]:
FMT = '%Y-%m-%d %H:%M:%S'

INITIAL_LATITUDE = 41.477182778
INITIAL_LONGITUDE = -74.916578
FINAL_LATITUDE = 40.129715978
FINAL_LONGITUDE = -73.120778

In [ ]:
def Trips_agg(listA, listB):
    listA.extend(listB)
    
    return(listA)

In [ ]:
def Max_trips(listA):
    listA.sort(key = lambda x: x[4], reverse = True)
    listA = listA[:10]
    listA = list(map(lambda line: [line[0], line[1], line[2], line[3]], listA))
    
    return(listA)

In [ ]:
try:
    lines = sc.textFile('sorted_data.csv')
    lines = lines.map(lambda line: line.split(',')) \
                 .filter(lambda entry: (float(entry[6]) >= INITIAL_LONGITUDE and \
                                        float(entry[6]) <= FINAL_LONGITUDE and \
                                        float(entry[8]) >= INITIAL_LONGITUDE and \
                                        float(entry[8]) <= FINAL_LONGITUDE and \
                                        float(entry[7]) >= FINAL_LATITUDE and \
                                        float(entry[7]) <= INITIAL_LATITUDE and \
                                        float(entry[9]) >= FINAL_LATITUDE and \
                                        float(entry[9]) <= INITIAL_LATITUDE)) \
                 .map(lambda entry: [datetime.datetime.strptime(entry[2], FMT).hour, \
                                     datetime.datetime.strptime(entry[3], FMT).hour, \
                                     entry[4], \
                                     builtins.round(float(entry[7]),2), \
                                     builtins.round(float(entry[6]),2), \
                                     builtins.round(float(entry[9]),2), \
                                     builtins.round(float(entry[8]),2), \
                                     entry[11], \
                                     entry[14], \
                                     datetime.datetime.strptime(entry[2], FMT).strftime('%A'), \
                                     1])
    
    lines = lines.filter(lambda line: (line[2] != 0 and line[0] == line[1] and ((line[3] != line[5]) or (line[4] != line[6]))))
    
    lines = lines.map(lambda line: ((line[9], line[0], line[3], line[4], line[5], line[6]), line[10]))
    lines = lines.reduceByKey(lambda trip_1, trip_2: trip_1 + trip_2)
    
    lines = lines.map(lambda line: ((line[0][0], line[0][1]), [(line[0][2], line[0][3], line[0][4], line[0][5], line[1])])) 
    lines = lines.reduceByKey(lambda trip_1, trip_2: Trips_agg(trip_1, trip_2))
    
    lines = lines.map(lambda line: [line[0][0], line[0][1], Max_trips(line[1])])
    
    for row in lines.collect():
        print(row)

    sc.stop()
    
except err:
    print(err)
    sc.stop()